### CR_2.11 - PaaS

In [15]:
#!/usr/bin/env python3
from pwn import remote, log
from math import gcd
from Crypto.Util.number import long_to_bytes, inverse

# Parametri noti
HOST = "paas.challs.cyberchallenge.it"
PORT = 9047
e = 65537

def get_encrypted_flag(r):
    r.recvuntil(b"What do you want to do?")
    r.sendline(b"1")
    r.recvuntil(b"N: ")
    n_flag = int(r.recvline().strip())
    r.recvuntil(b"Ciphertext: ")
    ct_flag = int(r.recvline().strip())
    log.info(f"Flag modulus: {n_flag}")
    return n_flag, ct_flag

def get_encryption(r, msg):
    r.recvuntil(b"What do you want to do?")
    r.sendline(b"2")
    r.recvuntil(b"> ")
    r.sendline(str(msg).encode())
    r.recvuntil(b"N: ")
    n = int(r.recvline().strip())
    r.recvuntil(b"Ciphertext: ")
    ct = int(r.recvline().strip())
    return n, ct

def main():
    # Connessione al server
    r = remote(HOST, PORT)

    # Raccogliamo una buona quantità di moduli da cifrature note
    collected = []
    NUM_QUERIES = 200  # puoi aumentare questo numero se necessario
    for i in range(NUM_QUERIES):
        n, _ = get_encryption(r, 2)  # il messaggio scelto non è rilevante
        collected.append(n)
        log.info(f"[{i+1}/{NUM_QUERIES}] n = {n}")
    
    # Richiediamo la cifratura della flag
    n_flag, ct_flag = get_encrypted_flag(r)
    
    # Cerca un fattore comune tra n_flag e uno dei moduli raccolti
    p = None
    for n in collected:
        g = gcd(n_flag, n)
        if g != 1 and g != n_flag:
            p = g
            log.success(f"Fattore comune trovato: {p}")
            break

    if not p:
        log.error("Nessun fattore comune trovato, riprova aumentando il numero di query.")
        r.close()
        return

    # Fattorizzazione del modulo della flag
    q = n_flag // p
    log.info(f"Fattori RSA: p = {p}, q = {q}")
    
    # Calcolo dell'esponente privato
    phi = (p - 1) * (q - 1)
    d = inverse(e, phi)
    
    # Decrittazione della flag
    m = pow(ct_flag, d, n_flag)
    flag = long_to_bytes(m)
    log.success(f"Flag: {flag.decode()}")

    r.close()

if __name__ == "__main__":
    main()


[x] Opening connection to paas.challs.cyberchallenge.it on port 9047
[x] Opening connection to paas.challs.cyberchallenge.it on port 9047: Trying 5.75.221.48
[+] Opening connection to paas.challs.cyberchallenge.it on port 9047: Done
[*] [1/200] n = 135509864232814554563068126994557581797177386761959440216066119210890401670055346290473074880266581001785085931378673152981082778311409026316291635186131175459772269358320058357349977436452858368136805383668850942803305615440632250329053462811201030694439306012960567462045297208350263775340594157456238502091
[*] [2/200] n = 73346001490957611823435027575449696043553273222887751209466489732038813382171438674582708240347094038550074228649396544560418210285638810323464260886991333123056356459861933161777678176684076556664449582100249575540408356557760567876161887221760125288643868747418664143096564374735013903774431301116508157719
[*] [3/200] n = 1376282330365620398820601699848483098960767237866754579396253275350705661520646826468704350879049019